In [ ]:
import cv2

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import numpy as np

In [ ]:
video_path = "/path/to/video"

In [ ]:
cap = cv2.VideoCapture(video_path)

video = []

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    video.append(frame)

cap.release()

In [ ]:
len(video)

In [ ]:
plt.imshow(video[0])

In [ ]:
# calculate optical flow

prev_frame = cv2.cvtColor(video[0], cv2.COLOR_BGR2GRAY)
flow = []
for next_frame in video[1:]:
    next_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    flow.append(
        cv2.calcOpticalFlowFarneback(
            prev_frame, next_frame, None, 0.5, 3, 32, 3, 5, 1.2, 0
        )
    )
    prev_frame = next_frame

In [ ]:
def track_point(flow, track_point, win_sz=10):
    track_point = np.array(track_point, dtype=np.float64)
    trajectory = [track_point]

    for i, ff in enumerate(flow):
        y, x = (
            int(track_point[1]),
            int(track_point[0]),
        )
        shift = ff[y : y + win_sz, x : x + win_sz, :]
        shift = np.mean(shift, axis=(0, 1))
        track_point = track_point + shift
        trajectory.append(track_point)

    return trajectory

In [ ]:
p0 = np.array((600, 1000))
trajectory_p0 = track_point(flow, p0)

In [ ]:
p1 = np.array((640, 850))
trajectory_p1 = track_point(flow, p1)

In [ ]:
for i in [0, len(trajectory_p0) - 1]:
    img = cv2.cvtColor(cv2.cvtColor(video[i], cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)
    for p in [trajectory_p0, trajectory_p1]:
        img = cv2.circle(img, p[i].astype(np.int32), 10, (255, 0, 0), thickness=-1)
    plt.imshow(img)
    plt.show()

In [ ]:
len(trajectory_p1)

In [ ]:
len(video)

In [ ]:
def gamma_correction(src, gamma):
    img = src / 256.0
    img = img**gamma
    img = img * 256.0
    return img

In [ ]:
gamma = 2.2

In [ ]:
final_image = gamma_correction(video[0].astype(np.float64), gamma=1 / gamma)
denominator = np.ones(video[0].shape[:2], dtype=np.float64)
mask = np.ones(video[0].shape[:2], dtype=np.float64)

for i in range(1, len(video)):
    h, w, _ = video[i].shape
    m, _ = cv2.estimateAffinePartial2D(
        np.stack([p0, p1]), np.stack([trajectory_p0[i], trajectory_p1[i]])
    )
    img_tr = cv2.warpAffine(
        gamma_correction(video[i], gamma=1 / gamma), m, dsize=(w, h)
    )
    final_image += img_tr.astype(np.float64)
    mask_tr = cv2.warpAffine(mask, m, dsize=(w, h))
    denominator += mask_tr

In [ ]:
img_tr.min(), img_tr.max()

In [ ]:
v = gamma_correction(img_tr.astype(np.float64), gamma=1 / gamma)
v.min(), v.max()

In [ ]:
final_image = final_image / denominator[..., np.newaxis]

In [ ]:
final_image.min(), final_image.max()

In [ ]:
final_image = gamma_correction(final_image, gamma=gamma)

In [ ]:
final_image = final_image / 256.0

In [ ]:
# convert to 16 bit uint image
final_image = final_image * 256 * 256
final_image = final_image.astype(np.uint16)

In [ ]:
cv2.imwrite("log_exposure.tiff", final_image)

In [ ]:
plt.imshow(final_image)